# Data Preparation

This notebook creates adds new data from other files new features (feature-engineering), clean up the data by deleting rows with missing values and

- Aggregate data from external data sources
- Feature engineering
- Data cleaning

In [52]:
# Libraries
import numpy as np
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Aggregate data from external data sources

The data is originaly form the modul data analytics. The first part is to clean up the data and aggregate the data with other sources.

The main idea originates from data analytics week 3.

### Import the data

In [53]:
# Read the data to a pandas data frame
df_ki = pd.read_csv('apartments_data_zurich_30.12.2023_with_bfs.csv', sep=',', encoding='utf-8')
df_da = pd.read_csv('apartments_data_zurich_with_bfs.csv', sep=',', encoding='utf-8')

#Drops the columns text_raw
df_da.drop(['text_raw'], axis=1, inplace=True)

# merges the two data frames together
df = pd.concat([df_ki, df_da], ignore_index=True)

# Get number of rows and columns
df.shape

(1725, 8)

### Add municipality data

In [54]:
# Meaning of variables:
# bfs_number: official municipality id
# bfs_name: official municipality name
# pop: number of residents (=population)
# pop_dens: population density (pop per km2)
# frg_pct: percentage foreigners
# emp: numer of employees

df_municip = pd.read_excel('municipality_data.xlsx', 
                           sheet_name='data_for_import')
df_municip.head(5)

,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0
2,3,Bonstetten,5572,749.932705,16.564968,1014.0
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0
4,5,Hedingen,3778,578.560490,16.410799,1478.0


In [55]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df = df.merge(df_municip[['bfs_number', 
                            'bfs_name',
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp']], 
                on="bfs_number")
df.head(5)

,bfs_number,rooms,area,price,postalcode,address,town,description_raw,bfs_name,pop,pop_dens,frg_pct,emp
0,261,4.5,148,4180,8050,"Schaffhauserstrasse 363, 8050 Zürich",Zürich,««Renovierte 4.5-Zimmerwohnung an zentraler La...,Zürich,420217,4778.994655,32.458468,491193.0
1,261,2.0,122,3190,8050,"Max Bill Platz 5, 8050 Zürich",Zürich,«Modernes Wohnen im Zentrum von Oerlikon»,Zürich,420217,4778.994655,32.458468,491193.0
2,261,3.5,78,2780,8050,"Regensbergstrasse 30, 8050 Zürich",Zürich,«Wohnen am Puls von Zürich»,Zürich,420217,4778.994655,32.458468,491193.0
3,261,3.5,69,3750,8050,"Dörflistrasse 112, 8050 Zürich",Zürich,"«Trendy Oerlikon 3.5 Room w/ Balcony, 1min to ...",Zürich,420217,4778.994655,32.458468,491193.0
4,261,3.5,74,2390,8050,"Schaffhauserstrasse 445, 8050 Zürich",Zürich,«Helle und zentrale Wohnung in Zürich-Oerlikon»,Zürich,420217,4778.994655,32.458468,491193.0


In [56]:
df.shape

(1725, 13)

### Add tax incom per municipal

In [57]:

df_tax_income = pd.read_csv('steuerbares_einkommen_2017.csv',
                            sep=',', encoding='utf-8', 
                            header=0, names=['bfs_number', 'bfs_name', 'yearly_income_in_mio', 'tax_income'])
df_tax_income.head(5)

,bfs_number,bfs_name,yearly_income_in_mio,tax_income
0,1,Aeugst am Albis,98,108'788
1,2,Affoltern am Albis,391,72'583
2,3,Bonstetten,224,91'002
3,4,Hausen am Albis,148,91'766
4,5,Hedingen,155,94'456


In [58]:
df_tax_income['tax_income'] = df_tax_income['tax_income'].str.replace("'", "").astype(int)

In [59]:
df.head(4)

,bfs_number,rooms,area,price,postalcode,address,town,description_raw,bfs_name,pop,pop_dens,frg_pct,emp
0,261,4.5,148,4180,8050,"Schaffhauserstrasse 363, 8050 Zürich",Zürich,««Renovierte 4.5-Zimmerwohnung an zentraler La...,Zürich,420217,4778.994655,32.458468,491193.0
1,261,2.0,122,3190,8050,"Max Bill Platz 5, 8050 Zürich",Zürich,«Modernes Wohnen im Zentrum von Oerlikon»,Zürich,420217,4778.994655,32.458468,491193.0
2,261,3.5,78,2780,8050,"Regensbergstrasse 30, 8050 Zürich",Zürich,«Wohnen am Puls von Zürich»,Zürich,420217,4778.994655,32.458468,491193.0
3,261,3.5,69,3750,8050,"Dörflistrasse 112, 8050 Zürich",Zürich,"«Trendy Oerlikon 3.5 Room w/ Balcony, 1min to ...",Zürich,420217,4778.994655,32.458468,491193.0


In [60]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df = df.merge(df_tax_income[['bfs_number', 
                            'tax_income']], 
                on="bfs_number")
df.head(5)

,bfs_number,rooms,area,price,postalcode,address,town,description_raw,bfs_name,pop,pop_dens,frg_pct,emp,tax_income
0,261,4.5,148,4180,8050,"Schaffhauserstrasse 363, 8050 Zürich",Zürich,««Renovierte 4.5-Zimmerwohnung an zentraler La...,Zürich,420217,4778.994655,32.458468,491193.0,85446
1,261,2.0,122,3190,8050,"Max Bill Platz 5, 8050 Zürich",Zürich,«Modernes Wohnen im Zentrum von Oerlikon»,Zürich,420217,4778.994655,32.458468,491193.0,85446
2,261,3.5,78,2780,8050,"Regensbergstrasse 30, 8050 Zürich",Zürich,«Wohnen am Puls von Zürich»,Zürich,420217,4778.994655,32.458468,491193.0,85446
3,261,3.5,69,3750,8050,"Dörflistrasse 112, 8050 Zürich",Zürich,"«Trendy Oerlikon 3.5 Room w/ Balcony, 1min to ...",Zürich,420217,4778.994655,32.458468,491193.0,85446
4,261,3.5,74,2390,8050,"Schaffhauserstrasse 445, 8050 Zürich",Zürich,«Helle und zentrale Wohnung in Zürich-Oerlikon»,Zürich,420217,4778.994655,32.458468,491193.0,85446


### Count number of rows and columns in the data frame

In [61]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (1680, 14)
Number of rows: 1680
Number of columns: 14


## Feature engineering

### Create additional variables from the apartment's descriptions

#### Create new binary (0/1) variable 'luxurious'

In [62]:
# Create a pattern which can be used to search the variable 'description_raw'
pattern = '(LOFT)|(SEESICHT)|(ATTIKA)|(LUXURIÖS)|(POOL)|(EXKLUSIV)'

# Create new variable 'luxurious' as binary dummy (0/1) variable
df['luxurious'] = df['description_raw'].str.contains(pat = pattern, case=False).astype(int)
print(df['luxurious'].sum())

# Show values
df[['description_raw','rooms','area','price','luxurious']]

119


,description_raw,rooms,area,price,luxurious
0,««Renovierte 4.5-Zimmerwohnung an zentraler La...,4.5,148,4180,0
1,«Modernes Wohnen im Zentrum von Oerlikon»,2.0,122,3190,0
2,«Wohnen am Puls von Zürich»,3.5,78,2780,0
3,"«Trendy Oerlikon 3.5 Room w/ Balcony, 1min to ...",3.5,69,3750,0
4,«Helle und zentrale Wohnung in Zürich-Oerlikon»,3.5,74,2390,0
...,...,...,...,...,...
1675,«Top moderne Wohnung an ruhiger Lage»,2.0,48,1490,0
1676,«Schöne Wohnung an ruhiger Lage»,3.5,79,1700,0
1677,«Ein glücklicher Wohnungsstart»,2.5,64,1770,0
1678,«Auf zwei Etagen verteilt»,3.5,80,1800,0


In [63]:
# Create a pattern which can be used to search the variable 'description_raw'
pattern = '(LOFT)|(SEESICHT)|(ATTIKA)|(LUXURIÖS)|(POOL)|(EXKLUSIV)'
prefix = 'lux_cat_'


# create empty 
for single_lux_cat in pattern.split('|'):
    df[prefix + single_lux_cat] = 0

for single_lux_cat in pattern.split('|'):
    # adds the value of the single luxurious category in the columns 'luxurious_cat'
    df[prefix + single_lux_cat] = df['description_raw'].str.contains(pat = pattern, case=False).astype(int)
    


In [64]:
df.describe()

,bfs_number,rooms,area,price,postalcode,pop,pop_dens,frg_pct,emp,tax_income,luxurious,lux_cat_(LOFT),lux_cat_(SEESICHT),lux_cat_(ATTIKA),lux_cat_(LUXURIÖS),lux_cat_(POOL),lux_cat_(EXKLUSIV)
count,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1672.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000,1680.000000
mean,188.332143,3.242262,83.613095,2580.466667,8311.620238,162041.540476,2560.932706,28.481929,180401.575359,83600.307143,0.070833,0.070833,0.070833,0.070833,0.070833,0.070833,0.070833
std,80.873421,1.130655,34.808046,1204.717634,289.958085,188289.046358,1737.551601,7.340503,225613.367458,17985.240107,0.256623,0.256623,0.256623,0.256623,0.256623,0.256623,0.256623
min,2.000000,1.000000,12.000000,16.000000,8001.000000,577.000000,82.310984,7.547170,129.000000,65147.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,118.000000,2.500000,62.000000,1812.250000,8049.000000,10280.000000,1161.756374,24.535004,5053.000000,72919.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,230.000000,3.500000,80.000000,2250.000000,8193.000000,24990.000000,1662.597326,29.819021,19226.000000,85446.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,261.000000,4.000000,100.000000,3000.000000,8600.000000,420217.000000,4778.994655,32.458468,491193.000000,85446.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,298.000000,9.000000,300.000000,9950.000000,8955.000000,420217.000000,4778.994655,46.226483,491193.000000,193412.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Create new categorical variable based on apartment area

In [65]:
labels = ['0 - 49', '50 - 99', '100 - 500']
df["area_cat"] = pd.cut(df.area, bins=[0, 50, 100, 500], labels=labels)
df[['area', 'area_cat']].head(10)
# area	area_cat
# 0	122	100 - 500
# 1	87	50 - 99
# 2	92	50 - 99

,area,area_cat
0,148,100 - 500
1,122,100 - 500
2,78,50 - 99
3,69,50 - 99
4,74,50 - 99
5,98,50 - 99
6,60,50 - 99
7,64,50 - 99
8,92,50 - 99
9,85,50 - 99


In [66]:
from sklearn.preprocessing import OrdinalEncoder

area_cat = df[['area_cat']]
ordinal_encoding = OrdinalEncoder()
area_cat_encoded = ordinal_encoding.fit_transform(area_cat) 
print(ordinal_encoding.categories_) # [array(['0 - 49', '100 - 500', '50 - 99'], dtype=object)]
print(np.unique(area_cat_encoded))  # [0. 1. 2.]
print(area_cat_encoded[:5])
df['area_cat_ordinal_ecoded'] = area_cat_encoded

[array(['0 - 49', '100 - 500', '50 - 99'], dtype=object)]
[0. 1. 2.]
[[1.]
 [1.]
 [2.]
 [2.]
 [2.]]


#### Create new numeric variable 'price_per_m2' and 'm2_per_rooms'

In [67]:
# Create the new variable
df['price_per_m2'] = round(df['price'] / df['area'], 2)

# Create the new variable
df['m2_per_rooms'] = round(df['area'] / df['rooms'] , 2)
# description_raw	rooms	area	area_cat	price	luxurious	price_per_m2
# 0	«Grosse Galerie, Terrasse mit Pergola, Berg- u...	3.5	122	100 - 500	3180	0	26.07

# Show values
df[['description_raw','rooms','area', 'area_cat','price','luxurious', 'price_per_m2']]

,description_raw,rooms,area,area_cat,price,luxurious,price_per_m2
0,««Renovierte 4.5-Zimmerwohnung an zentraler La...,4.5,148,100 - 500,4180,0,28.24
1,«Modernes Wohnen im Zentrum von Oerlikon»,2.0,122,100 - 500,3190,0,26.15
2,«Wohnen am Puls von Zürich»,3.5,78,50 - 99,2780,0,35.64
3,"«Trendy Oerlikon 3.5 Room w/ Balcony, 1min to ...",3.5,69,50 - 99,3750,0,54.35
4,«Helle und zentrale Wohnung in Zürich-Oerlikon»,3.5,74,50 - 99,2390,0,32.30
...,...,...,...,...,...,...,...
1675,«Top moderne Wohnung an ruhiger Lage»,2.0,48,0 - 49,1490,0,31.04
1676,«Schöne Wohnung an ruhiger Lage»,3.5,79,50 - 99,1700,0,21.52
1677,«Ein glücklicher Wohnungsstart»,2.5,64,50 - 99,1770,0,27.66
1678,«Auf zwei Etagen verteilt»,3.5,80,50 - 99,1800,0,22.50


## Data cleaning

### Count, identify and remove missing values

In [68]:
# Count missing values
print('Count missing values per variable')
print(pd.isna(df).sum(), '\n')

# Identify rows with missing values
print('Identify rows with missing values')
print(df.loc[df.isna().any(axis=1)][['rooms', 'area', 'price', 'address', 'emp']], '\n')

# Drop rows where at least one element is missing.
df2 = df.dropna()
df2.head()

Count missing values per variable
bfs_number                  0
rooms                       0
area                        0
price                       0
postalcode                  0
address                    50
town                        0
description_raw             0
bfs_name                    0
pop                         0
pop_dens                    0
frg_pct                     0
emp                         8
tax_income                  0
luxurious                   0
lux_cat_(LOFT)              0
lux_cat_(SEESICHT)          0
lux_cat_(ATTIKA)            0
lux_cat_(LUXURIÖS)          0
lux_cat_(POOL)              0
lux_cat_(EXKLUSIV)          0
area_cat                    0
area_cat_ordinal_ecoded     0
price_per_m2                0
m2_per_rooms                0
dtype: int64 

Identify rows with missing values
      rooms  area  price               address       emp
390     3.5   110   2880                   NaN  491193.0
391     5.0    80   2900                   NaN  49119

,bfs_number,rooms,area,price,postalcode,address,town,description_raw,bfs_name,pop,...,lux_cat_(LOFT),lux_cat_(SEESICHT),lux_cat_(ATTIKA),lux_cat_(LUXURIÖS),lux_cat_(POOL),lux_cat_(EXKLUSIV),area_cat,area_cat_ordinal_ecoded,price_per_m2,m2_per_rooms
0,261,4.5,148,4180,8050,"Schaffhauserstrasse 363, 8050 Zürich",Zürich,««Renovierte 4.5-Zimmerwohnung an zentraler La...,Zürich,420217,...,0,0,0,0,0,0,100 - 500,1.0,28.24,32.89
1,261,2.0,122,3190,8050,"Max Bill Platz 5, 8050 Zürich",Zürich,«Modernes Wohnen im Zentrum von Oerlikon»,Zürich,420217,...,0,0,0,0,0,0,100 - 500,1.0,26.15,61.00
2,261,3.5,78,2780,8050,"Regensbergstrasse 30, 8050 Zürich",Zürich,«Wohnen am Puls von Zürich»,Zürich,420217,...,0,0,0,0,0,0,50 - 99,2.0,35.64,22.29
3,261,3.5,69,3750,8050,"Dörflistrasse 112, 8050 Zürich",Zürich,"«Trendy Oerlikon 3.5 Room w/ Balcony, 1min to ...",Zürich,420217,...,0,0,0,0,0,0,50 - 99,2.0,54.35,19.71
4,261,3.5,74,2390,8050,"Schaffhauserstrasse 445, 8050 Zürich",Zürich,«Helle und zentrale Wohnung in Zürich-Oerlikon»,Zürich,420217,...,0,0,0,0,0,0,50 - 99,2.0,32.30,21.14


### Count, identify & remove duplicated values

In [69]:
# Count duplicated values in the whole data set
print('Sum of missing values:', df.duplicated().sum(), '\n')

# Identify duplicated values in 'rooms', 'area', 'price'
print('Duplicated values')
print(df.loc[df.duplicated(keep = 'last')])

# Drop the rows with duplicated values
df3 = df2.drop_duplicates()

Sum of missing values: 44 

Duplicated values
      bfs_number  rooms  area  price  postalcode  \
20           261    1.5    24   1960        8050   
23           261    1.5    23   1900        8048   
44           261    1.5    25   3490        8008   
205          261    3.5    60   3000        8037   
206          261    1.0    45   2300        8037   
207          261    1.0    29   2150        8037   
211          261    1.0    36   2150        8037   
212          261    1.0    35   2250        8037   
214          261    3.5    60   3000        8037   
215          261    2.5    70   3100        8037   
216          261    1.0    34   2200        8037   
217          261    1.0    29   2150        8037   
218          261    1.0    36   2150        8037   
219          261    2.5    59   2850        8037   
220          261    1.0    29   2150        8037   
221          261    1.0    34   2200        8037   
222          261    1.0    36   2150        8037   
223          261  

#### Save data to file

In [70]:
df3.columns

Index(['bfs_number', 'rooms', 'area', 'price', 'postalcode', 'address', 'town',
       'description_raw', 'bfs_name', 'pop', 'pop_dens', 'frg_pct', 'emp',
       'tax_income', 'luxurious', 'lux_cat_(LOFT)', 'lux_cat_(SEESICHT)',
       'lux_cat_(ATTIKA)', 'lux_cat_(LUXURIÖS)', 'lux_cat_(POOL)',
       'lux_cat_(EXKLUSIV)', 'area_cat', 'area_cat_ordinal_ecoded',
       'price_per_m2', 'm2_per_rooms'],
      dtype='object')

In [71]:
df3.to_csv('apartments_data_enriched.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)